In [ ]:
def kyle_print(string):
    print(string)

In [ ]:
import matplotlib as mpl
from PIL import ImageColor

#Alex's code.
def color_fader(cs, mix=0):
    if len(cs) == 2:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
    elif len(cs) == 3:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        c3 = np.array(mpl.colors.to_rgb(cs[2]))
        if mix <= .5:
            mix *= 2 # create 0-1 range
            return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
        else:
            mix -= .5 # create 0-1 range
            mix *= 2
            return mpl.colors.to_hex((1-mix)*c2 + mix*c3)

def create_color_buckets(colors=['#FF0000', '#00FF00'], num_buckets=10, display=True): #RGB
    color_buckets = []

    if display:
        fig, ax = plt.subplots(figsize=(4, 2))
    
    for x in range(0, num_buckets):
        X = color_fader(colors, x/(num_buckets-1))
        
        if display:
            ax.axvline(x, color=X, linewidth=4)
        
        color_buckets.append(ImageColor.getcolor(X, "RGB")[::-1]) # convert RGB to BGR
        
    if display:
        plt.axis('off')
        plt.show()

    return color_buckets

In [ ]:
import copy
import cv2
import imutils
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import ImageColor
from skimage import exposure
from skimage.measure import label, regionprops
from skimage.morphology import area_opening, disk, square
import sys

def scoregrass(image1, sensitivity_list):  
    hsi_images1 = []
    hsv1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HSV)
    hsi_images1.append(hsv1)
    imgreturn = hsv1
    hue_template = np.zeros(hsv1.shape[0:2])
    
    for sensitivity in sensitivity_list:
        #where is hue within 60+-sensitivity
        lower = np.array([60 - sensitivity]).astype(np.uint8)
        upper = np.array([60+sensitivity]).astype(np.uint8)
        mask = cv2.inRange(hsv1[:,:,0], lower, upper).astype(bool)
        hue_template[mask] = hue_template[mask]+1
        '''for i in range(0, hsv1.shape[0]): #iterate over y position
            for j in range(0, hsv1.shape[1]): # iterate over x position   
                hue = hsv1[i][j][0]
                if(hue in range(60-sensitivity,60+sensitivity)): #look at this 60 value in more detail.
                    score_return[i][j] = score_return[i][j] + 1
                imgreturn = hsv1'''
    imgreturn = hsv1
    return imgreturn,hue_template      
    # define range of blue color in HSV
    #lower_green = np.array([60 - sensitivity, 100, 50])
    #upper_green = np.array([60 + sensitivity, 255, 255])
    # Threshold the HSV image to get only blue colors
    #mask1 = cv2.inRange(hsv1, lower_green, upper_green)
    # Bitwise-AND mask and original image
    #res1 = cv2.bitwise_and(image1,image1, mask= mask1)
    #cv2.imshow('image1',image1)
    #cv2.imshow('mask1',mask1)
    #cv2.imshow('res1',res1)
    


'''def absolute_image_differencing_with_HSL(image1,image2,threshold):
    #First convert the image to HSL
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HLS)
    image2 = cv2.cvtColor(image2,cv2.COLOR_BGR2HLS)
    print(image1)
    #get only the hue channel
    image1hues = image1[:,:,0]
    image2hues = image2[:,:,0]
    for each in image1hues:
        print(each)
    return 0

image_differences = []
'''



In [ ]:
def setup_buckets(num_buckets_val):
    color_buckets = create_color_buckets(colors=['#FF0000', '#00FF00'], num_buckets=num_buckets_val, display=False)
    color_buckets.insert(0,(0,0,0))
#     print(color_buckets)
    return color_buckets, len(color_buckets)

def score_missions(regions,num_buckets_val, heatmap_anomaly_scores=[0], heatmap_sensitivity_min=15, heatmap_sensitivity_max=37):
    print('Anomaly Detection 3 + Grass Heatmap')
    
    # ---- create anomaly list, determine location of non grass region
    raw_anomaly_masks = [] # list of masks where each mask is anomalies for that mission
    grass_region_mask = (regions[0] == [0, 0, 0]) # find black regions
    grass_region_mask = np.logical_not(grass_region_mask[:,  :, 0]).astype(bool) # invert, all channels are the same, grab one so that dimensions are (543, 470) and compatible with other masks
    # ----
    
    #regions are just the grass regions.
    #total_grass_region_pixels = np.sum(regions[0] != [0,0,0]) #don't include black pixels.
    mask2 = [] #setup a mask two to hold everything for scores 0 through 1 for now.
    anomaly_detections = []
    blackpixels = np.sum(regions[0] == [0,0,0])/3 #include black pixels
    color_buckets, num_buckets_val = setup_buckets(num_buckets_val)
#     print("size color buckets is " + str(len(color_buckets)) + " and numbuckets is " + str(num_buckets_val))
    grass_differences = []
    percent_buckets = [] #list of lists. #return the percentages for every image that are in each color.
    mask_list = []
    hwlength = regions[0].shape[0]*regions[0].shape[1]
    blackpercent = np.divide(blackpixels,hwlength)
    for i in range(0,len(regions)):#score every mission image
        current_bucket_percent_list = []
        image1 = copy.deepcopy(regions[i])
        #score_return = np.zeros((image1.shape[0:2])) #creates
            #rad1=5 
            #rad2=10
            #rad3=17
        linspace_values = np.linspace(heatmap_sensitivity_min, heatmap_sensitivity_max, num_buckets_val-1).astype(int)
#         print("linspace values: ",linspace_values)
        score_return = scoregrass(image1,linspace_values)[1] #try num_buckets different sensitivities.
            #score_return2 = scoregrass(image1,rad1)
        var = copy.deepcopy(score_return)
        #var = var*25  #use this is we're using grayscale to better visualize changes between black and white
        color_score = np.zeros((image1.shape))
        tempvar = []
        mask2 = []
        
        # ---- mask will be updated with each score
        temp_anomaly_mask = np.zeros((regions[0].shape[0:2])).astype(bool)
        # ----
        
        for score in range(0, num_buckets_val):
            mask = score_return == score #a mask that gets all pixels that equal to our current score
            if i==0:
                mask_list.append(np.multiply(mask,255).astype(np.uint8))
            color_score[mask, :] = color_buckets[score] #color_score is our heatmap image.
            if score==0:
                mask2 = mask
#                 print("black score: " , np.sum(mask))
                val = (np.sum(mask)-blackpixels)/(hwlength-blackpixels)*100
            else:
                if score<3:
                    mask2 = np.logical_or(mask2, mask) #we may need to input a score list so that we don't hardcode 0 through 1.
                val = np.multiply(np.divide(np.sum(mask),hwlength-blackpixels),100)
            
            # ---- only include scores in list
            if score in heatmap_anomaly_scores:
                temp_anomaly_mask = np.logical_or(temp_anomaly_mask, mask.astype(bool))
            # ----
                
#             print("hwlength " , hwlength)
#             print("blaclength " , blackpixels)
            current_bucket_percent_list.append(val)  
            
        # ---- apply all scores
        temp_anomaly_mask = np.logical_and(temp_anomaly_mask, grass_region_mask)
        temp_anomaly_mask = temp_anomaly_mask.astype(int) # True/False --> 1/0
        temp_anomaly_mask = (temp_anomaly_mask*255)
        raw_anomaly_masks.append(temp_anomaly_mask)
        # ----
        
        variable = copy.deepcopy(color_score)
        grass_differences.append(variable.astype(np.uint8))        #print(i)             #imgDifference = absolute_image_differencing_with_HSL(image1,image2,25)             #image_differences.append(imgDifference)
        percent_buckets.append(current_bucket_percent_list)
        anomaly_detections.append(np.multiply(mask2,255).astype(np.uint8)) 
        
    # ---- subtract masks to get difference
    differenced_anomaly_masks = []
    for i in range(0, len(raw_anomaly_masks)-1):
        diff = np.abs(raw_anomaly_masks[i+1] - raw_anomaly_masks[i]).astype(np.uint8)
        result = cv2.erode(diff, disk(radius=7), iterations=1)
        result = cv2.dilate(result, disk(radius=10), iterations=1)
        differenced_anomaly_masks.append(result.astype(np.uint8))
    # ----
        
#     # ----
#     grass_region_mask = grass_region_mask.astype(int)
#     grass_region_mask = (grass_region_mask*255).astype(np.uint8)
#     # ----
    
    print('Done')
    
    return grass_differences, percent_buckets, mask_list, color_buckets, anomaly_detections, raw_anomaly_masks, differenced_anomaly_masks

In [ ]:
def findgrass(image1,sensitivity):
    hsv1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HSV)
    lower = np.array([60 - sensitivity]).astype(np.uint8)
    upper = np.array([60+sensitivity]).astype(np.uint8)
    mask = cv2.inRange(hsv1[:,:,0], lower, upper).astype(bool)
    mask = np.multiply(mask,255).astype(np.uint8)
    return mask   

In [ ]:
from skimage.morphology import disk, square


def anomaly_detection(imageList,sensitivity):
    print('Anomaly Detection 2')
    hsi_image_differences = []
    for each in imageList:
        hsi_image_differences.append(findgrass(each,sensitivity))
    masks = anomaly_detection_helper(hsi_image_differences)
    print('Done')
    return masks

def anomaly_detection_helper(imageList):
    counter = 0
    retList = []
    prevImage = copy.deepcopy(imageList[0])
    for eachImg in imageList:
        if counter==0:
            prevImage = copy.deepcopy(eachImg)
            counter+=1
            continue
        differenced = copy.deepcopy(eachImg-prevImage)
        prevImage = copy.deepcopy(eachImg) #update previous image
        retList.append(np.abs(differenced))
        counter+=1
    #return retList
    #feed the return list into the eroding functions
    masks = []
    for img in retList:
        res = cv2.erode(img, disk(radius=1), iterations=1)# get rid of small stuff #square 4 was best so far.
        mask = cv2.dilate(img,disk(radius=7),iterations=1) #look into disk and square morphologies.
        masks.append(mask)
    return masks

In [ ]:
#make interactive plot
import clickplot as cp
def make_interactive_plot(aplot):
    plot = cp(aplot)
    plot.show()